In [ ]:
!git clone https://github.com/serengil/deepface.git
%cd deepface
!pip install -e .

In [ ]:
from google.colab import drive
import zipfile
import os

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Rutas de tus archivos ZIP (modifica los nombres según los reales)
zips = [
    "/content/drive/MyDrive/TFG_Instagram/fotos_filtradas_LPGA.zip",
    "/content/drive/MyDrive/TFG_Instagram/FotosPerfilLPGA.zip"
]

# 3. Carpeta de destino base en /content
base_destino = "/content/"
os.makedirs(base_destino, exist_ok=True)

# 4. Extraer cada ZIP en su propia subcarpeta
for zip_path in zips:
    nombre = os.path.splitext(os.path.basename(zip_path))[0]
    destino = os.path.join(base_destino, nombre)
    os.makedirs(destino, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(destino)
        print("Extraído")


In [ ]:
from PIL import Image
import os

# Ruta de la carpeta a analizar
carpeta = "/content/publicaciones_2024"  # cámbiala si es necesario

# Recorremos todos los archivos
for archivo in os.listdir(carpeta):
    if archivo.lower().endswith((".jpg")):
        ruta = os.path.join(carpeta, archivo)
        try:
            with Image.open(ruta) as img:
                img.verify()  # Verifica si la imagen es válida
        except Exception as e:
            print(f"Imagen corrupta eliminada: {archivo} → {e}")
            os.remove(ruta)

In [ ]:
from PIL import Image
import os

# Ruta a tus imágenes de referencia
carpeta_referencias = "/content/referencias"

# Lista para guardar los nombres incorrectos
webp_falsos = []

# Paso 1: Detectar imágenes que son .webp pero con extensión .jpg
for archivo in os.listdir(carpeta_referencias):
    if archivo.lower().endswith(".jpg"):
        ruta = os.path.join(carpeta_referencias, archivo)
        try:
            with Image.open(ruta) as img:
                if img.format.lower() == "webp":
                    webp_falsos.append(archivo)
        except Exception as e:
            print(f"Error al abrir {archivo}: {e}")

print(f"\nSe detectaron {len(webp_falsos)} archivos .webp con extensión .jpg.")

# Paso 2: Convertir a JPG correctamente
for archivo in webp_falsos:
    ruta = os.path.join(carpeta_referencias, archivo)
    try:
        with Image.open(ruta).convert("RGB") as img:
            img.save(ruta, format="JPEG")
            print(f"Convertido correctamente: {archivo}")
    except Exception as e:
        print(f"Error al convertir {archivo}: {e}")

In [ ]:
from deepface import DeepFace
import pandas as pd
import os

# Configuración
modelo = "ArcFace"
detector = "retinaface"
umbral_match = 0.68

# Rutas
carpeta_imagenes = "/content/fotos_filtradas_LPGA"
carpeta_referencias = "/content/FotosPerfilLPGA"
output_path = "/content/drive/MyDrive/TFG_Instagram/predicciones_jugadora_por_imagen_con_boxes_LPGA.xlsx"

# Listado de imágenes y jugadores
imagenes_dataset = [f for f in os.listdir(carpeta_imagenes) if f.lower().endswith(".jpg")]
jugadores = [f[:-4] for f in os.listdir(carpeta_referencias) if f.lower().endswith(".jpg")]

# Inicializar dataframe
df_resultados = pd.DataFrame()
df_resultados["imagen"] = imagenes_dataset

# Comparación jugador por jugador
for ref_file in os.listdir(carpeta_referencias):
    if not ref_file.lower().endswith(".jpg"):
        continue

    jugador = os.path.splitext(ref_file)[0]
    ref_path = os.path.join(carpeta_referencias, ref_file)

    print(f"Buscando a {jugador} en todas las publicaciones...")

    try:
        resultado = DeepFace.find(
            img_path=ref_path,
            db_path=carpeta_imagenes,
            model_name=modelo,
            detector_backend=detector,
            enforce_detection=False
        )
        df_find = resultado[0] if resultado and not resultado[0].empty else pd.DataFrame()
    except Exception as e:
        print(f"Error con {jugador}: {e}")
        df_find = pd.DataFrame()

    # Inicializar columnas del jugador
    detecciones = {img: 0 for img in imagenes_dataset}
    distancias = {img: 5.0 for img in imagenes_dataset}
    boxes = {img: None for img in imagenes_dataset}

    # Procesar resultados
    if not df_find.empty:
        for _, row in df_find.iterrows():
            img_name = os.path.basename(row["identity"])
            distancia = row["distance"]
            if distancia > umbral_match or img_name not in detecciones:
                continue
            if distancia < distancias[img_name]:
                detecciones[img_name] = 1
                distancias[img_name] = distancia
                try:
                    box = (
                        int(row["target_x"]),
                        int(row["target_y"]),
                        int(row["target_w"]),
                        int(row["target_h"])
                    )
                except:
                    box = None
                boxes[img_name] = box

    # Añadir columnas al DataFrame
    df_resultados[jugador] = df_resultados["imagen"].apply(lambda x: detecciones[x])
    df_resultados[f"dist_{jugador}"] = df_resultados["imagen"].apply(lambda x: distancias[x])
    df_resultados[f"box_{jugador}"] = df_resultados["imagen"].apply(lambda x: boxes[x])

# Guardar CSV
df_resultados.to_excel(output_path, index=False)
print(f"Resultados guardados en: {output_path}")




In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
# === 1. Cargar archivo Excel con predicciones ===
ruta_excel = "/content/drive/MyDrive/TFG_Instagram/predicciones_jugadora_por_imagen_con_boxes_LPGA.xlsx"
df = pd.read_excel(ruta_excel)

# === 2. Identificar jugadores ===
jugadores = [col for col in df.columns if not col.startswith(("dist_", "box_", "imagen"))]

# === 3. Inicializar una copia del DataFrame ===
df_filtrado = df.copy()

# === 4. Procesar fila por fila para filtrar por menor distancia en cada box ===
for idx, row in df.iterrows():
    box_to_jugadores = {}

    for jugador in jugadores:
        box = row.get(f"box_{jugador}")
        if pd.notna(box) and row[jugador] == 1:
            box_to_jugadores.setdefault(box, []).append(jugador)

    for box, jug_group in box_to_jugadores.items():
        if len(jug_group) > 1:
            distancias = {jug: row[f"dist_{jug}"] for jug in jug_group}
            mejor = min(distancias, key=distancias.get)
            for jug in jug_group:
                if jug != mejor:
                    df_filtrado.at[idx, jug] = 0

# === 5. Eliminar columnas de distancia y box ===
cols_distancia = [col for col in df_filtrado.columns if col.startswith("dist_")]
cols_boxes = [col for col in df_filtrado.columns if col.startswith("box_")]
df_final = df_filtrado.drop(columns=cols_distancia + cols_boxes)

# === 6. Guardar resultado (opcional) ===
salida = "/content/drive/MyDrive/TFG_Instagram/predicciones_jugadora_por_imagen_limpio_LPGA.csv"
df_final.to_csv(salida, index=False, sep=';')
print(f"Archivo final guardado en: {salida}")

In [ ]:
import pandas as pd

# Cargar el archivo CSV de predicciones
ruta_csv = "/content/drive/MyDrive/TFG_Instagram/predicciones_jugador_por_imagen_limpio.csv"
df = pd.read_csv(ruta_csv, sep=";", low_memory=False)

# Crear una columna con el nombre del propietario de cada imagen (extraído del nombre de archivo)
df["propietario"] = df["imagen"].apply(lambda x: x.split("_")[0].lower())

# Obtener lista de jugadores
jugadores = [col for col in df.columns if col not in ["imagen", "propietario"]]

# Inicializar resultados
estadisticas = []

for jugador in jugadores:
    total = df[df[jugador] == 1].shape[0]
    en_sus_fotos = df[(df["propietario"] == jugador) & (df[jugador] == 1)].shape[0]
    en_fotos_de_otros = total - en_sus_fotos

    estadisticas.append({
        "Jugador": jugador,
        "Total apariciones": total,
        "Propias": en_sus_fotos,
        "Ajenas": en_fotos_de_otros
    })

# Crear DataFrame con resultados
df_stats = pd.DataFrame(estadisticas).sort_values(by="Total apariciones", ascending=False)

# Guardar si quieres
df_stats.to_csv("/content/drive/MyDrive/TFG_Instagram/apariciones_por_jugador.csv", index=False, sep=";")

print("Resumen guardado en /content/drive/MyDrive/TFG_Instagram/apariciones_por_jugador.csv")


In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

ruta_csv = "/content/drive/MyDrive/TFG_Instagram/predicciones_jugadora_por_imagen_limpio_LPGA.csv"
directorio_imagenes = "/content/fotos_filtradas_LPGA"
jugador_objetivo = "nellykorda"

df = pd.read_csv(ruta_csv, sep=";", low_memory=False)

# Verificar que el jugador existe
if jugador_objetivo not in df.columns:
    print(f"Jugador '{jugador_objetivo}' no encontrado en el archivo.")
else:
    # Filtrar las imágenes donde aparece
    imagenes = df[df[jugador_objetivo] == 1]

    print(f"{len(imagenes)} imágenes donde aparece {jugador_objetivo}")


    for _, row in imagenes.iterrows():
        nombre_imagen = row["imagen"]
        ruta = os.path.join(directorio_imagenes, nombre_imagen)

        if not os.path.exists(ruta):
            print(f"Imagen no encontrada: {ruta}")
            continue

        # Cargar y mostrar la imagen
        img = cv2.imread(ruta)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis("off")
        plt.title(nombre_imagen)
        plt.show()

        # Obtener otros jugadores detectados
        jugadores_detectados = [col for col in df.columns if col not in ["imagen", "propietario"] and row[col] == 1]
        print(f"Personas detectadas: {', '.join(jugadores_detectados)}\n")


***PRUEBAS***

In [ ]:
from deepface import DeepFace
import pandas as pd
import os

# Configuración
modelo = "ArcFace"
detector = "retinaface"
umbral_match = 0.68

# Rutas
carpeta_imagenes = "/content/shane"
carpeta_referencias = "/content/referencias"
output_path = f"/content/drive/MyDrive/TFG_Instagram/Shane.xlsx"

# Listado de imágenes y jugadores
imagenes_dataset = [f for f in os.listdir(carpeta_imagenes) if f.lower().endswith(".jpg")]
jugadores = [f[:-4] for f in os.listdir(carpeta_referencias) if f.lower().endswith(".jpg")]

# Inicializar dataframe
df_resultados = pd.DataFrame()
df_resultados["imagen"] = imagenes_dataset

# Comparación jugador por jugador
for ref_file in os.listdir(carpeta_referencias):
    if not ref_file.lower().endswith(".jpg"):
        continue

    jugador = os.path.splitext(ref_file)[0]
    ref_path = os.path.join(carpeta_referencias, ref_file)

    print(f"Buscando a {jugador} en todas las publicaciones...")

    try:
        resultado = DeepFace.find(
            img_path=ref_path,
            db_path=carpeta_imagenes,
            model_name=modelo,
            detector_backend=detector,
            enforce_detection=False
        )
        df_find = resultado[0] if resultado and not resultado[0].empty else pd.DataFrame()
    except Exception as e:
        print(f"Error con {jugador}: {e}")
        df_find = pd.DataFrame()

    # Inicializar columnas del jugador
    detecciones = {img: 0 for img in imagenes_dataset}
    distancias = {img: 5.0 for img in imagenes_dataset}
    boxes = {img: None for img in imagenes_dataset}

    # Procesar resultados
    if not df_find.empty:
        for _, row in df_find.iterrows():
            img_name = os.path.basename(row["identity"])
            distancia = row["distance"]
            if distancia > umbral_match or img_name not in detecciones:
                continue
            if distancia < distancias[img_name]:
                detecciones[img_name] = 1
                distancias[img_name] = distancia
                try:
                    box = (
                        int(row["target_x"]),
                        int(row["target_y"]),
                        int(row["target_w"]),
                        int(row["target_h"])
                    )
                except:
                    box = None
                boxes[img_name] = box

    # Añadir columnas al DataFrame
    df_resultados[jugador] = df_resultados["imagen"].apply(lambda x: detecciones[x])
    df_resultados[f"dist_{jugador}"] = df_resultados["imagen"].apply(lambda x: distancias[x])
    df_resultados[f"box_{jugador}"] = df_resultados["imagen"].apply(lambda x: boxes[x])

# Guardar CSV
df_resultados.to_excel(output_path, index=False)
print(f"Resultados guardados en: {output_path}")


In [ ]:
from deepface import DeepFace
import pandas as pd
import os
import shutil
from google.colab import drive


drive.mount('/content/drive')


ground_truth_path = "/content/drive/MyDrive/TFG_PruebasModelos/ground_truth_jugadores_nombrados.xlsx"
df_gt = pd.read_excel(ground_truth_path)
imagenes_dataset = df_gt["imagen"].tolist()

origen_drive = "/content/drive/MyDrive/TFG_PruebasModelos/Jugadores/"
db_local = "/content/deepface_db_524"
os.makedirs(db_local, exist_ok=True)

if not os.listdir(db_local):  # copiar solo si está vacío
    for img in imagenes_dataset:
        src = os.path.join(origen_drive, img)
        dst = os.path.join(db_local, img)
        if os.path.exists(src):
            shutil.copy2(src, dst)


jugadores = {
    "joaquin_niemann": "/content/drive/MyDrive/TFG_PruebasModelos/CarasTOP/joaquin_niemann/joaquin.jpg",
    "jon_rahm": "/content/drive/MyDrive/TFG_PruebasModelos/CarasTOP/jon_rahm/rahm.jpg",
    "scottie_scheffler": "/content/drive/MyDrive/TFG_PruebasModelos/CarasTOP/scottie_scheffler/scottie.jpg",
    "rory_mcilroy": "/content/drive/MyDrive/TFG_PruebasModelos/CarasTOP/rory_mcilroy/rory.jpg"
}


modelo = "ArcFace"
detector = "retinaface"
umbral_match = 0.68
output_path = f"/content/drive/MyDrive/TFG_PruebasModelos/PruebaConBox.xlsx"

df_resultados = pd.DataFrame()
df_resultados["imagen"] = imagenes_dataset


for jugador, ref_path in jugadores.items():
    print(f"\nBuscando a {jugador}...")

    try:
        resultado = DeepFace.find(
            img_path=ref_path,
            db_path=db_local,
            model_name=modelo,
            detector_backend=detector,
            enforce_detection=False,
            threshold=umbral_match
        )
        df_find = resultado[0] if resultado and not resultado[0].empty else pd.DataFrame()

    except Exception as e:
        print(f"Error con {jugador}: {e}")
        df_find = pd.DataFrame()

    # Inicializar columnas vacías por imagen
    detecciones = {img: 0 for img in imagenes_dataset}
    distancias = {img: 5.0 for img in imagenes_dataset}
    boxes = {img: None for img in imagenes_dataset}

    if not df_find.empty:
        for _, row in df_find.iterrows():
            img_name = os.path.basename(row["identity"])
            distancia = row["distance"]
            if distancia > umbral_match or img_name not in detecciones:
                continue
            if distancia < distancias[img_name]:
                detecciones[img_name] = 1
                distancias[img_name] = distancia
                try:
                    box = (
                        int(row["target_x"]),
                        int(row["target_y"]),
                        int(row["target_w"]),
                        int(row["target_h"])
                    )
                except:
                    box = None
                boxes[img_name] = box

    # Añadir columnas al DataFrame
    df_resultados[jugador] = df_resultados["imagen"].apply(lambda x: detecciones[x])
    df_resultados[f"dist_{jugador}"] = df_resultados["imagen"].apply(lambda x: distancias[x])
    df_resultados[f"box_{jugador}"] = df_resultados["imagen"].apply(lambda x: boxes[x])

df_resultados.to_excel(output_path, index=False)
print(f"Resultados guardados en: {output_path}")


In [ ]:
import pandas as pd

# Cargar archivo Excel con predicciones
ruta_excel = "/content/drive/MyDrive/TFG_PruebasModelos/PruebaConBox.xlsx"
df = pd.read_excel(ruta_excel)

# Identificar jugadores
jugadores = [col for col in df.columns if not col.startswith(("dist_", "box_", "imagen"))]

# Inicializar una copia del DataFrame
df_filtrado = df.copy()

# Procesar fila por fila para filtrar por menor distancia en cada box
for idx, row in df.iterrows():
    box_to_jugadores = {}

    for jugador in jugadores:
        box = row.get(f"box_{jugador}")
        if pd.notna(box) and row[jugador] == 1:
            box_to_jugadores.setdefault(box, []).append(jugador)

    for box, jug_group in box_to_jugadores.items():
        if len(jug_group) > 1:
            distancias = {jug: row[f"dist_{jug}"] for jug in jug_group}
            mejor = min(distancias, key=distancias.get)
            for jug in jug_group:
                if jug != mejor:
                    df_filtrado.at[idx, jug] = 0

# Eliminar columnas de distancia y box
cols_distancia = [col for col in df_filtrado.columns if col.startswith("dist_")]
cols_boxes = [col for col in df_filtrado.columns if col.startswith("box_")]
df_final = df_filtrado.drop(columns=cols_distancia + cols_boxes)

# Guardar resultado (opcional)
salida = "/content/resultado_final_sin_boxes.xlsx"
df_final.to_excel(salida, index=False)
print(f"Archivo final guardado en: {salida}")



In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from google.colab import drive


drive.mount('/content/drive')

ground_truth_path = "/content/drive/MyDrive/TFG_PruebasModelos/ground_truth_jugadores_nombrados.xlsx"
predicciones_path = "/content/resultado_final_sin_boxes.xlsx"
jugadores = ["joaco_niemann", "jonrahm", "scottie.scheffler", "rorymcilroy"]

# CARGAR DATOS
df_gt = pd.read_excel(ground_truth_path)
df_pred = pd.read_excel(predicciones_path)

df_merged = pd.merge(df_gt, df_pred, on="imagen", suffixes=("_gt", "_pred"))

# CALCULAR MÉTRICAS POR JUGADOR
resultados = []
TP_total = FP_total = FN_total = TN_total = 0

for jugador in jugadores:
    col_gt = f"{jugador}_gt"
    col_pred = f"{jugador}_pred"

    if col_gt not in df_merged.columns or col_pred not in df_merged.columns:
        continue

    y_true = df_merged[col_gt].astype(int)
    y_pred = df_merged[col_pred].astype(int)

    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc = (tp + tn) / (tp + tn + fp + fn)

    TP_total += tp
    FP_total += fp
    FN_total += fn
    TN_total += tn

    resultados.append({
        "jugador": jugador,
        "precision": round(prec, 4),
        "recall": round(rec, 4),
        "f1_score": round(f1, 4),
        "accuracy": round(acc, 4)
    })

# MÉTRICAS GLOBALES
prec_total = TP_total / (TP_total + FP_total) if (TP_total + FP_total) > 0 else 0
rec_total = TP_total / (TP_total + FN_total) if (TP_total + FN_total) > 0 else 0
f1_total = 2 * (prec_total * rec_total) / (prec_total + rec_total) if (prec_total + rec_total) > 0 else 0
acc_total = (TP_total + TN_total) / (TP_total + TN_total + FP_total + FN_total)

resultados.append({
    "jugador": "TOTAL",
    "precision": round(prec_total, 4),
    "recall": round(rec_total, 4),
    "f1_score": round(f1_total, 4),
    "accuracy": round(acc_total, 4)
})

# RESULTADO FINAL
df_resultados = pd.DataFrame(resultados)
print(df_resultados.to_markdown(index=False))